### Import Dependencies

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import variational
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.ticker import FuncFormatter
from itertools import cycle
import os
import time
import math
import pandas as pd
from collections import OrderedDict
from sklearn.linear_model import LogisticRegression
import copy
import torch.nn as nn
from torch.autograd import Variable
from typing import List
import itertools
from tqdm.autonotebook import tqdm
from models import *
import models
from logger import *
import wandb
from thirdparty.repdistiller.helper.util import adjust_learning_rate as sgda_adjust_learning_rate
from thirdparty.repdistiller.distiller_zoo import DistillKL, HintLoss, Attention, Similarity, Correlation, VIDLoss, RKDLoss
from thirdparty.repdistiller.distiller_zoo import PKT, ABLoss, FactorTransfer, KDSVD, FSP, NSTLoss
from thirdparty.repdistiller.helper.loops import train_distill, train_distill_hide, train_distill_linear, train_vanilla, train_negrad, train_bcu, train_bcu_distill, validate
from thirdparty.repdistiller.helper.pretrain import init
import os
os.environ['CUDA_DEVICE_ORDER']='PCI_BUS_ID'
os.environ["CUDA_VISIBLE_DEVICES"] = ""

/var/folders/mw/2gycpkq57p97cpwvmwvm69m00000gn/T/ipykernel_3251/2126072653.py:20: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


### Train Original Model

dataset: mnist
model: mlp
dataroot: data/MNIST
resume from standard_model_for_pilot.pt in checkpoints folder

In [ ]:
%run main.py --dataset mnist --model mlp --dataroot=data/MNIST/ --filters 1.0 --lr 0.001 \
--resume checkpoints/standard_model_for_pilot.pt --disable-bn \
--weight-decay 0.1 --batch-size 128 --epochs 31 --seed 3